In [3]:

#install opencv

import numpy as np
import cv2, time, pandas 
from datetime import datetime


movcount=0
# sample video
cap = cv2.VideoCapture('test2.mp4')

# Time of movement 
timelist = [] 


time_skips = float(2000) #skip every 2 seconds. You need to modify this


# reading back-to-back frames(images) from video

ret, frame1 = cap.read()
ret, frame2 = cap.read()
#time.sleep(5)

count = 0
while cap.isOpened():
    
    
    #cap.set(cv2.CAP_PROP_POS_MSEC,(count*time_skips))
    
    count += 1
    #print(count)
    #cv2.rectangle(frame1, (0, 0), (350, 350), (0, 255, 0), 2)
    
    # Difference between frame1(image) and frame2(image)
    diff = cv2.absdiff(frame1, frame2)
    # Converting color image to gray_scale image
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

    # Converting gray scale image to GaussianBlur, so that change can be find easily 
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # If pixel value is greater than 20, it is assigned white(255) otherwise black
    _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
    dilated = cv2.dilate(thresh, None, iterations=4)

    # finding contours of moving object
    contours, hirarchy = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # making rectangle around moving object
    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        # restrict the area (consider movements in a specific area)
        #if (0<(x+w)/2<200) and (0<(y+h)/2<200):
        if x:
            if (cv2.contourArea(contour) < 100):
                continue
            #print(datetime.now())  
            #(a.hour, a.minute, a.second)
            timelist.append(datetime.now())  
            movcount += 1
            cv2.rectangle(frame1, (x, y), (x+w, y+h), (0, 255, 255), 2)

    
    # Display original frame
    cv2.imshow('Motion Detector', frame1)

    # Display Diffrenciate Frame
    cv2.imshow('Difference Frame', thresh)

    # Assign frame2(image) to frame1(image)
    frame1 = frame2

    #Read new frame2
    ret, frame2 = cap.read()
    
    # Press 'esc' for quit 
    if cv2.waitKey(40) == 27:
        break
    
print(movcount)  

# Release cap resource
cap.release()

# Destroy all windows
cv2.destroyAllWindows()

KeyboardInterrupt: 